In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

In [2]:
from crewai import Agent, Crew, Task

In [3]:
from dotenv import load_dotenv
import os

load_dotenv()  # Load environment variables from .env

openai_api_key = os.getenv("OPENAI_API_KEY")
if openai_api_key is None:
    raise ValueError("OPENAI_API_KEY is not set. Please add it to your .env file.")

os.environ["OPENAI_API_KEY"] = openai_api_key  # Ensuring CrewAI uses the correct key
os.environ["OPENAI_MODEL_NAME"] = 'gpt-4o-mini'

In [4]:
# Step 1: Requirement Extraction Agent - Processes Call Transcripts
transcript_analyst = Agent(
    role="Call Transcript Analyst",
    goal="Extract customer requirements from call transcripts where staff discuss needs with customers.",
    backstory=(
        "An customer service agent specializing in analyzing call transcripts. It listens to conversations between staff and customers, "
        "identifies key requirements, and structures them into actionable insights for policy matching."
    ),
    allow_delegation=False,
    verbose=True
)

In [5]:
from pydantic import BaseModel
from typing import List, Optional
from datetime import date
class TravelInsuranceRequirement(BaseModel):
    """
    A Pydantic model representing travel insurance requirements extracted from customer conversations.
    
    This class structures and validates travel insurance requirements data, making it easier to process
    and analyze customer needs systematically. It captures essential details about travel plans,
    insurance preferences, and specific customer requirements.
    """
    requirement_id: str  # Unique identifier for tracking
    
    requirement_summary: str  # Summary of the customer's insurance needs
    detailed_description: str  # More detailed requirement extracted from transcript

    travel_destination: Optional[str]  # Country or region the user is traveling to
    travel_duration: Optional[str]  # e.g., "7 days", "1 month", "6 months"
    travel_start_date: Optional[date]  # Date when the travel begins
    travel_end_date: Optional[date]  # Date when the travel ends
    
    insurance_coverage_type: Optional[List[str]]  # e.g., ["Medical", "Trip Cancellation", "Baggage Loss"]
    pre_existing_conditions: Optional[List[str]]  # Any health conditions to be covered
    age_group: Optional[str]  # e.g., "18-25", "26-40", "41-60", "Senior"
    travelers_count: Optional[int]  # Number of travelers to be insured
    
    budget_range: Optional[str]  # e.g., "$50-$100", "$100-$200"
    preferred_insurance_provider: Optional[str]  # If the user has a preference
    
    additional_requests: Optional[str]  # Any special requests, like "Coverage for adventure sports"
    
    keywords: Optional[List[str]]  # Important keywords for analytics

In [6]:
# Define the agent task
transcript_analyst_task = Task(
    description="""Analyze the travel insurance call transcript below and extract key customer requirements including:
    1. Travel destination and dates
    2. Number of travelers and their ages
    3. Citizenship/residency status
    4. Desired coverage types and preferences
    5. Budget considerations
    6. Any special requests or concerns
    
    Transcript:
    {parsed_transcripts}
    """,
    expected_output="A structured summary of customer needs based on call transcripts, formatted for policy matching.",
    agent=transcript_analyst,
    output_json=TravelInsuranceRequirement,
    output_file="insurance_requirement.json"
)

In [7]:
# Define the crew with agents and tasks
insurance_recommendation_crew = Crew(
    agents=[transcript_analyst],
    tasks=[transcript_analyst_task],
    verbose=True
)

In [8]:
#transcript_file_path = "call_transcript1.txt"
#parsed_transcripts = parse_transcripts(transcript_file_path)

#input_transcript = {
#    'parsed_transcripts': parsed_transcripts
#}

In [9]:
import json
import uuid
from datetime import datetime

current_dir = os.getcwd()
project_root = os.path.dirname(os.path.dirname(os.path.dirname(current_dir)))

# Update the path to point to the JSON transcript
sample_transcript = os.path.join(project_root, "data", "processed_transcript", "parsed_transcript_05.json")

# Read and parse the JSON transcript
with open(sample_transcript, 'r') as f:
    transcript_data = json.load(f)

# Convert the transcript data to a string format for the agent
formatted_transcript = "\n".join([f"{msg['speaker']}: {msg['dialogue']}" for msg in transcript_data])

input_transcript = {
    'parsed_transcripts': formatted_transcript
}

In [10]:
result = insurance_recommendation_crew.kickoff(inputs=input_transcript)

 [DEBUG]: == Working Agent: Call Transcript Analyst
 [INFO]: == Starting Task: Analyze the travel insurance call transcript below and extract key customer requirements including:
    1. Travel destination and dates
    2. Number of travelers and their ages
    3. Citizenship/residency status
    4. Desired coverage types and preferences
    5. Budget considerations
    6. Any special requests or concerns

    Transcript:
    Customer Service Agent: Good morning, thank you for calling ISS Insurance, this is Sarah, how may I assist you today?
Customer: Good morning Sarah, ah, yes, hello. I’m calling to ask about your travel insurance. Thinking of going on a trip, you see, and want to be sure I get a good deal, you know? Travel insurance can be quite pricey these days.
Customer Service Agent: Certainly, I understand. We can definitely help you with information on our travel insurance plans. To start, could you tell me a bit about your travel plans? Where are you planning to travel to?
Cus

In [11]:
from pprint import pformat
from IPython.display import Markdown, display

pretty_dict = pformat(result, sort_dicts=False)
display(Markdown(f"```\n{pretty_dict}\n```"))

```
('{\n'
 '  "requirement_id": "1",\n'
 '  "transcript_file": "",\n'
 '  "requirement_summary": "Travel insurance for trip to Thailand.",\n'
 '  "detailed_description": "Customer traveling to Thailand for 10 days with '
 'spouse, looking for basic medical emergency and lost luggage coverage. '
 'Interested in a standard plan with competitive pricing and potential couple '
 'discounts.",\n'
 '  "travel_destination": "Thailand (Bangkok and Phuket)",\n'
 '  "travel_duration": "10 days",\n'
 '  "travel_start_date": "2023-11-15",\n'
 '  "travel_end_date": "2023-11-25",\n'
 '  "insurance_coverage_type": [\n'
 '    "basic medical emergency",\n'
 '    "lost luggage"\n'
 '  ],\n'
 '  "pre_existing_conditions": null,\n'
 '  "age_group": "45-42",\n'
 '  "travelers_count": 2,\n'
 '  "budget_range": "competitive pricing",\n'
 '  "preferred_insurance_provider": null,\n'
 '  "pain_points": [\n'
 '    "concern about higher rates due to age",\n'
 '    "need for best quote comparison"\n'
 '  ],\n'
 '  "additional_requests": "inquire about discounts for couples",\n'
 '  "keywords": [\n'
 '    "Thailand",\n'
 '    "travel insurance",\n'
 '    "couple discounts",\n'
 '    "competitive pricing"\n'
 '  ]\n'
 '}')
```